In [3]:
import pandas as pd
import duckdb

import plotly.express as px

In [4]:
# Caminho do banco
db_path = "../data/duckdb/database.duckdb"

# Conexão com o banco DuckDB
con = duckdb.connect(db_path)

# Carrega os dados de feature engineering
df = con.execute("""
    SELECT 
          CG.*
        , CC.cluster
                 
    FROM gold.consumo_geral AS CG
    
    INNER JOIN output.clusterizacao_cliente AS CC ON
        CG.client_id = CC.client_id
""").df()

df

,date,client_id,region,temperature,humidity,consumption_kwh,cluster
0,2023-01-01,C0006,Centro,25.652941,57.188235,17.47,1
1,2023-01-01,C0007,Centro,25.652941,57.188235,14.42,1
2,2023-01-01,C0008,Centro,25.652941,57.188235,17.59,1
3,2023-01-01,C0009,Centro,25.652941,57.188235,14.63,4
4,2023-01-01,C0011,Centro,25.652941,57.188235,12.97,1
...,...,...,...,...,...,...,...
17095,2023-06-29,C0078,Sul,25.493333,65.840000,7.91,3
17096,2023-06-29,C0079,Sul,25.493333,65.840000,12.31,3
17097,2023-06-29,C0083,Sul,25.493333,65.840000,19.49,3
17098,2023-06-29,C0087,Sul,25.493333,65.840000,16.28,3


In [6]:
fig = px.histogram(df, x="cluster", color="cluster", barmode="group",
                   title="Distribuição de Clientes por Cluster")
fig.update_layout(xaxis_title="Cluster", yaxis_title="Número de Clientes")
fig.show()

In [ ]:
fig = px.box(df, x="cluster", y="consumption_kwh", color="cluster",
             title="Distribuição de Consumo por Cluster")
fig.update_layout(xaxis_title="Cluster", yaxis_title="Consumo (kWh)")
fig.show()

# Os consumos médio diário por cluster são fatores de separação entre os clusters.

In [ ]:
region_counts = df.groupby(["region", "cluster"]).size().reset_index(name='count')

fig = px.bar(region_counts, x='region', y='count', color='cluster', barmode='group',
             title="Distribuição de Clientes por Região e Cluster")
fig.update_layout(xaxis_title="Região", yaxis_title="Número de Clientes")
fig.show()

# As regiões Centro e Oeste tem característica semelhantes.
# As regiões Leste e Norte tem característica semelhantes.

In [ ]:
df['date'] = pd.to_datetime(df['date'])  # Garante que seja datetime

consumo_diario = df.groupby(['date', 'cluster'])['consumption_kwh'].mean().reset_index()

fig = px.line(consumo_diario, x='date', y='consumption_kwh', color='cluster',
              title='Consumo Médio Diário por Cluster')
fig.update_layout(xaxis_title="Data", yaxis_title="Consumo Médio (kWh)")
fig.show()

# Os clusters 1 e 2 apresentam um padrão de consumo mais elevado ao longo do tempo.
# Os clusters 0 e 4 apresentam um padrão de consumo mais baixo.
# O cluster 3 tem um padrão de consumo intermediário.

In [13]:
fig = px.histogram(df, x="temperature", color="cluster", barmode="overlay",
                   marginal="box", opacity=0.6,
                   title="Distribuição da Temperatura por Cluster")

fig.update_layout(
    xaxis_title="Temperatura (°C)",
    yaxis_title="Frequência",
    template="simple_white"
)
fig.show()


# O cluster 3 Não tem faixa de temperatura definida.
# Os clusters 1 e 4 tem faixa de temperatura semelhantes.
# Os clusters 0 e 2 tem faixa de temperatura semelhantes.

In [ ]:
fig = px.histogram(df, x="humidity", color="cluster", barmode="overlay",
                   marginal="box", opacity=0.6,
                   title="Distribuição da Umidade por Cluster")
fig.update_layout(
    xaxis_title="Umidade (%)",
    yaxis_title="Frequência",
    template="simple_white"
)
fig.show()

# O cluster 3 Não tem faixa de umidade definida.
# Os clusters 1 e 4 tem faixa de umidade semelhantes.
# Os clusters 0 e 2 tem faixa de umidade semelhantes.

# Insights

### Padrões de Consumo por Cluster

- Clusters 1 e 2 apresentam os maiores consumos médios diários, indicando grupos com clientes de maior demanda energética.
- Clusters 0 e 4 têm os menores consumos, sugerindo clientes com comportamento mais econômico ou perfil residencial menor.
- Cluster 3 está em um nível intermediário de consumo, podendo representar um perfil de cliente de classe média ou padrão misto.

### Perfil Climático por Cluster

- Cluster 3 não possui faixa climática (temperatura e umidade) bem definida porém se concentra na região Sul.
- Clusters 1 e 4 compartilham faixas semelhantes de temperatura e umidade, o que pode indicar atuação em regiões com clima semelhante sendo Centro e Oeste.
- O mesmo ocorre entre os clusters 0 e 2, que também compartilham perfis climáticos similares sendo Norte e Leste.

### Potenciais Aplicações

- Personalização de ofertas: Clientes dos clusters 1 e 2 podem ser foco de campanhas de eficiência energética, painéis solares ou tarifas especiais.
- Planejamento regional: Identificar onde estão concentrados clusters com alta demanda pode ajudar na alocação de infraestrutura elétrica.
- Segmentação climática: As faixas de clima ajudam a entender se o consumo é mais impactado por temperatura/umidade ou por outros fatores (como tamanho da residência ou perfil de uso).